<a href="https://colab.research.google.com/github/Aerospace87/ML-projects/blob/main/tensorflow/Text_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount your Google Drive to the local /content/drive directory
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from os import listdir
with open("/content/drive/My Drive/dataset/shakespeare.txt") as f:
  shakespeare_dataset = f.read()

In [12]:
shakespeare_dataset = shakespeare_dataset.split('\n\n')
shakespeare_dataset = [[document] for document in shakespeare_dataset if document != ""]

In [13]:
shakespeare_dataset = shakespeare_dataset[2:-1]
print(len(shakespeare_dataset))

141


In [14]:
#from tensorflow import data
#shakespeare_dataset = data.Dataset.from_tensor_slices(shakespeare_dataset)

In [15]:
from tensorflow.keras import layers
text_vectorization = layers.TextVectorization(
    standardize='lower',
    split='whitespace',
    output_mode='int'
)


In [16]:
text_vectorization.adapt(shakespeare_dataset)

In [17]:
shakespeare_dataset = text_vectorization(shakespeare_dataset)

In [25]:
vocabulary_size = text_vectorization.vocabulary_size()

In [26]:
word_emedding_vector_dimension = 64
word_embedding = layers.Embedding(input_dim=vocabulary_size, output_dim=word_emedding_vector_dimension)

In [27]:
lstm_layer = layers.LSTM(128)

In [29]:
#from tensorflow.keras import Sequential

#steps = [text_vectorization, lstm_layer, lstm_layer]
#model = Sequential(steps)
#model.summary()



ValueError: Exception encountered when calling layer "text_vectorization_1" (type TextVectorization).

When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, None) with rank=2

Call arguments received by layer "text_vectorization_1" (type TextVectorization):
  • inputs=tf.Tensor(shape=(None, None), dtype=string)

# Import necessary libraries

In [ ]:
import collections
import pathlib

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

# Download and explore the dataset

In [ ]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

6053168/6053168 [==============================] - 0s 0us/step


In [ ]:
# Exploring teh content of the folder dataet_dir
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz'),
 PosixPath('/tmp/.keras/train'),
 PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/README.md')]

In [ ]:
# Defining the training set folder and exploring the content
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/java'),
 PosixPath('/tmp/.keras/train/python'),
 PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript')]

The folders /train/java, /train/python, /train/csharp and /train/javascript contains files representing a stack overflow question:

In [ ]:
train_java_dir = train_dir/"java"
list(train_java_dir.iterdir())

[PosixPath('/tmp/.keras/train/java/848.txt'),
 PosixPath('/tmp/.keras/train/java/1586.txt'),
 PosixPath('/tmp/.keras/train/java/1345.txt'),
 PosixPath('/tmp/.keras/train/java/1973.txt'),
 PosixPath('/tmp/.keras/train/java/1859.txt'),
 PosixPath('/tmp/.keras/train/java/1295.txt'),
 PosixPath('/tmp/.keras/train/java/1307.txt'),
 PosixPath('/tmp/.keras/train/java/1802.txt'),
 PosixPath('/tmp/.keras/train/java/502.txt'),
 PosixPath('/tmp/.keras/train/java/1172.txt'),
 PosixPath('/tmp/.keras/train/java/1861.txt'),
 PosixPath('/tmp/.keras/train/java/1513.txt'),
 PosixPath('/tmp/.keras/train/java/349.txt'),
 PosixPath('/tmp/.keras/train/java/1005.txt'),
 PosixPath('/tmp/.keras/train/java/1680.txt'),
 PosixPath('/tmp/.keras/train/java/550.txt'),
 PosixPath('/tmp/.keras/train/java/625.txt'),
 PosixPath('/tmp/.keras/train/java/791.txt'),
 PosixPath('/tmp/.keras/train/java/1022.txt'),
 PosixPath('/tmp/.keras/train/java/703.txt'),
 PosixPath('/tmp/.keras/train/java/940.txt'),
 PosixPath('/tmp/.ker

In [ ]:
train_java_dir_sample = train_java_dir/"848.txt"


with open(train_java_dir_sample) as f:
  print(f.read())

"look for an element in a 2d array using contains i have a 2d array and want to use contains method to check if mychar is part of myarray[i][0]..public static final string[][] myarray = {.        {""nm1"", ""desc1""},.        {""nm2"", ""desc2""},.        {""nm3"", ""desc3""},.        {""nm4"", ""desc4""},.        {""nm5"", ""desc5""}.    }..arrays.aslist(myarray).contains(mychar);...however this is failing for me. when i make it as 1d array like: ..public static final string[][] myarraynm = {.    ""nm1"",.    ""nm2"",.    ""nm3"",.    ""nm4"",.    ""nm5"".}..arrays.aslist(myarraynm).contains(mychar);...is working for me. is there a way i can do the value check using 2d array (loop will be the last option)"



## Loading the dataset

The tf.keras.utils.text_dataset_from_directory expects that the files are placed in folders representing each class and the name of the file is of the format number.txt as 1.txt: ecause a test set is already aaulle we define a validation set using 20% of samples in the training set which is shuffled randomly before samplings:


In [ ]:
# Training dataset
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [ ]:
# Validation dataset
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [ ]:
# Test dataset
test_dir = dataset_dir/'test'

raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [ ]:
raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_test_ds = raw_test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)